 * @ Author: Yohei Ohto
 * @ Create Time: 2025-11-28 20:10:30
 * @ Modified time: 2025-11-28 20:15:46
 * @ Description: 既存MLMへのNER実行

In [1]:
import glob
import re
import os

import evaluate
import numpy as np
import pandas as pd
import plotly.express as px
import torch
import wandb
from datasets import load_dataset
from sklearn.manifold import TSNE
from tqdm.auto import tqdm
from transformers import (AutoModel, AutoModelForTokenClassification,
                          AutoTokenizer, DataCollatorWithPadding, Trainer, DataCollatorForTokenClassification,
                          TrainingArguments)

import sys
sys.path.append('..')
from src import *

/home/ubuntu/test_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models = [
    "microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext",
    "dmis-lab/biobert-v1.1",
    "google-bert/bert-base-cased",
    "answerdotai/ModernBERT-base",
    "Simonlee711/Clinical_ModernBERT",
    "thomas-sounack/BioClinical-ModernBERT-base",
]

# NCBI Desease Named Entity Recognition Benchmark

In [3]:
dataset = load_dataset("ncbi/ncbi_disease")

label_names = [
    "O", "B-DISEASE", "I-DISEASE"
]

id2label = {i: name for i, name in enumerate(label_names)}
label2id = {name: i for i, name in enumerate(label_names)}

In [6]:
def compute_metrics(p: tuple):
    """
    NERの評価指標を計算する関数
    Args:
        p: モデルの予測結果とラベルのタプル (predictions, labels)
        label_names (list): ラベル名のリスト
    Returns:
        dict: precision, recall, f1, accuracyを含む辞書
    How to use:
        from transformers import Trainer

        trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        )
        
    """
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    seqeval = evaluate.load("seqeval")
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    
    return {
        "precision": float(results["overall_precision"]),
        "recall": float(results["overall_recall"]),
        "f1": float(results["overall_f1"]),
        "accuracy": float(results["overall_accuracy"]),
    }

In [10]:
for i, name in enumerate(models):
    print("=== Model:", name, " Training ===")
    
    tokenizer = AutoTokenizer.from_pretrained(name)
    tokenizer.pad_token = "[PAD]" 

    model = AutoModelForTokenClassification.from_pretrained(
    name,
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
    )
    
    tokenized_datasets = dataset.map(
        lambda x: tokenize_and_align_labels(x, tokenizer),
        batched=True
    )
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

    model_name = name.split("/")[-1]
    
    wandb.init(
    entity="250502_ohto_research",
    project="NCBI-disease", name=model_name, 
    config={
        "model_name": model_name,
        "learning_rate": 2e-5,
        "batch_size": 16,
        "num_epochs": 10,
        "dataset": "NCBI-Disease",
    })

    os.makedirs(f"../result/ncbi/{model_name}", exist_ok=True)

    training_args = TrainingArguments(
        output_dir=f"../result/ncbi/{model_name}",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        save_strategy="no",
        load_best_model_at_end=False,
        weight_decay=0.01,
        eval_strategy="epoch",
        report_to="wandb",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    metrics = trainer.evaluate()
    trainer.save_metrics("all", metrics)

    df_final_results = result_output_general(
        trainer=trainer,
        tokenized_datasets=tokenized_datasets,
        tokenizer=tokenizer,
        id2label=id2label,
        num_samples_to_process=len(tokenized_datasets["validation"]),
        output_filename=f'../result/ncbi/{model_name}/results.csv'
    )

=== Model: microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext  Training ===


Some weights of BertForTokenClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


eval/accuracy,▁▆▅▆▆▆█▇███
eval/f1,▁▆▅▆▇▇█▇█▇▇
eval/loss,▄▁▂▄▆▆▇█▇██
eval/precision,▁▅▅▅█▇▇▇█▇▇
eval/recall,▁▇▅█▄▇█▇█▇▇
eval/runtime,▂▁▄▂▂▁▂▃█▅▂
eval/samples_per_second,▇█▅▇▇█▇▆▁▄▇
eval/steps_per_second,▇█▅▇▇█▇▆▁▄▇
test/accuracy,▁
test/f1,▁
+11,...


/tmp/ipykernel_843539/3916018160.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.058861,0.799257,0.819568,0.809285,0.983270
2,0.070800,0.049048,0.831902,0.861499,0.846442,0.986733
3,0.019500,0.051331,0.847666,0.876747,0.861961,0.987025
4,0.019500,0.063080,0.841596,0.884371,0.862454,0.986691
5,0.008500,0.079091,0.855879,0.860229,0.858048,0.986649
6,0.003700,0.075670,0.864492,0.875476,0.869949,0.987692
7,0.003700,0.076995,0.862069,0.889454,0.875547,0.988318
8,0.001700,0.087122,0.854637,0.866582,0.860568,0.986900
9,0.001100,0.090444,0.852399,0.880559,0.866250,0.987192
10,0.001100,0.091009,0.855721,0.874206,0.864865,0.987192


...

=== Model: thomas-sounack/BioClinical-ModernBERT-base  Training ===


Some weights of ModernBertForTokenClassification were not initialized from the model checkpoint at thomas-sounack/BioClinical-ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 941/941 [00:00<00:00, 2543.63 examples/s]


eval/accuracy,▁▇▄▇▇▇█████
eval/f1,▁▄▁▇▅▆▇████
eval/loss,▁▁▄▄▇▆▇▇███
eval/precision,▁▃▄▄▅▄▇████
eval/recall,▂▆▁█▅▇▇▇▇▇▇
eval/runtime,▃▅▅▁▃▅▂▄▅█▄
eval/samples_per_second,▆▄▄█▆▄▇▅▄▁▅
eval/steps_per_second,▆▄▄█▆▄▇▅▄▁▅
test/accuracy,▁
test/f1,▁
+11,...


/tmp/ipykernel_843539/3916018160.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.054914,0.736397,0.773825,0.754647,0.981476
2,0.064000,0.050489,0.777778,0.809403,0.793275,0.983312
3,0.014300,0.055113,0.807018,0.818297,0.812618,0.984605
4,0.014300,0.068060,0.823745,0.855146,0.839152,0.985815
5,0.004000,0.089371,0.788666,0.848793,0.817625,0.984647
6,0.001000,0.089996,0.821208,0.846252,0.833542,0.985481
7,0.001000,0.094864,0.818841,0.861499,0.839628,0.985523
8,0.000200,0.098919,0.827086,0.869123,0.847584,0.985898
9,0.000100,0.100658,0.820913,0.867853,0.843731,0.985898
10,0.000100,0.101472,0.822892,0.867853,0.844774,0.985857


# BC5CDR

In [11]:
dataset = load_dataset("omniquad/BC5CDR-IOB")

label_names = [
    "O", "B-DISEASE", "I-DISEASE", "B-CHEMICAL", "I-CHEMICAL"
]

id2label = {i: name for i, name in enumerate(label_names)}
label2id = {name: i for i, name in enumerate(label_names)}

print("Label names:", label_names)
print("ID to Label mapping:", id2label)
print("Label to ID mapping:", label2id)

Generating test split: 100%|██████████| 4798/4798 [00:00<00:00, 57601.76 examples/s]

Label names: ['O', 'B-DISEASE', 'I-DISEASE', 'B-CHEMICAL', 'I-CHEMICAL']
ID to Label mapping: {0: 'O', 1: 'B-DISEASE', 2: 'I-DISEASE', 3: 'B-CHEMICAL', 4: 'I-CHEMICAL'}
Label to ID mapping: {'O': 0, 'B-DISEASE': 1, 'I-DISEASE': 2, 'B-CHEMICAL': 3, 'I-CHEMICAL': 4}


In [12]:
def convert_labels_to_ids(example):
    example['ner_tags'] = [label2id[label_str] for label_str in example['ner_tags']]
    return example
dataset = dataset.map(convert_labels_to_ids)

Map: 100%|██████████| 4798/4798 [00:00<00:00, 35685.00 examples/s]


In [13]:
for i, name in enumerate(models):
    print("=== Model:", name, " Training ===")
    
    tokenizer = AutoTokenizer.from_pretrained(name)
    tokenizer.pad_token = "[PAD]" 

    model = AutoModelForTokenClassification.from_pretrained(
    name,
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
    )
    
    tokenized_datasets = dataset.map(
        lambda x: tokenize_and_align_labels(x, tokenizer),
        batched=True
    )
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

    model_name = name.split("/")[-1]
    
    wandb.init(
    entity="250502_ohto_research",
    project="BC5", name=model_name, 
    config={
        "model_name": model_name,
        "learning_rate": 2e-5,
        "batch_size": 16,
        "num_epochs": 10,
        "dataset": "BC5",
    })

    os.makedirs(f"../result/bc5/{model_name}", exist_ok=True)

    training_args = TrainingArguments(
        output_dir=f"../result/bc5/{model_name}",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=10,
        save_strategy="no",
        load_best_model_at_end=False,
        weight_decay=0.01,
        eval_strategy="epoch",
        report_to="wandb",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    metrics = trainer.evaluate()
    trainer.save_metrics("all", metrics)

    df_final_results = result_output_general(
        trainer=trainer,
        tokenized_datasets=tokenized_datasets,
        tokenizer=tokenizer,
        id2label=id2label,
        num_samples_to_process=len(tokenized_datasets["validation"]),
        output_filename=f'../result/bc5/{model_name}/results.csv'
    )

=== Model: microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext  Training ===


Some weights of BertForTokenClassification were not initialized from the model checkpoint at microsoft/BiomedNLP-BiomedBERT-base-uncased-abstract-fulltext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 4798/4798 [00:00<00:00, 9819.01 examples/s]


eval/accuracy,▁▄▆█▆▇▇████
eval/f1,▁▄▅▇▆▇▇████
eval/loss,▂▁▂▃▆▆▇████
eval/precision,▁▄▆█▅█▇████
eval/recall,▁▄▄▇▇▆▇████
eval/runtime,▃▄▁█▄▂▃▄▁▂▄
eval/samples_per_second,▆▅█▁▅▇▆▅█▆▄
eval/steps_per_second,▆▅█▁▅▇▆▅█▆▄
test/accuracy,▁
test/f1,▁
+11,...


/tmp/ipykernel_843539/1228510784.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.082857,0.847195,0.894089,0.870011,0.972678
2,0.121300,0.076229,0.876698,0.901282,0.888820,0.977804
3,0.121300,0.086503,0.878622,0.903992,0.891127,0.976365
4,0.030400,0.096066,0.895124,0.912853,0.903902,0.979098
5,0.030400,0.111860,0.863264,0.922026,0.891678,0.975190
6,0.013500,0.114379,0.888190,0.912540,0.900201,0.977923
7,0.006300,0.121600,0.893972,0.910560,0.902190,0.978144
8,0.006300,0.128511,0.892719,0.912540,0.902521,0.978630
9,0.003100,0.129583,0.895001,0.912540,0.903685,0.978766
10,0.003100,0.130246,0.895716,0.913270,0.904408,0.978970


...

=== Model: thomas-sounack/BioClinical-ModernBERT-base  Training ===


Some weights of ModernBertForTokenClassification were not initialized from the model checkpoint at thomas-sounack/BioClinical-ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 4798/4798 [00:00<00:00, 8699.81 examples/s] 


eval/accuracy,▁▄▅▆█▇█████
eval/f1,▁▄▆▇███████
eval/loss,▁▂▃▄▅▆▇▇███
eval/precision,▁▆▇▆▇▅▃▄█▇▇
eval/recall,▁▃▅▆▇███▆▇▇
eval/runtime,▁▄▂▆▂▁█▄▃▅▁
eval/samples_per_second,█▅▇▃▇█▁▅▆▄█
eval/steps_per_second,█▅▇▃▇█▁▅▆▄█
test/accuracy,▁
test/f1,▁
+11,...


/tmp/ipykernel_843539/1228510784.py:48: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': None, 'bos_token_id': None}.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.092396,0.813974,0.878219,0.844877,0.968529
2,0.101800,0.105089,0.859606,0.810760,0.834469,0.969397
3,0.101800,0.112331,0.841091,0.864769,0.852766,0.971918
4,0.017200,0.135920,0.859498,0.875091,0.867225,0.973835
5,0.017200,0.151826,0.855923,0.877698,0.866674,0.973562
6,0.003100,0.169289,0.868666,0.866854,0.867759,0.973971
7,0.000500,0.165084,0.852307,0.882077,0.866937,0.973920
8,0.000500,0.174699,0.862749,0.875613,0.869133,0.974065
9,0.000100,0.178495,0.861598,0.875613,0.868549,0.973980
10,0.000100,0.179900,0.861850,0.875508,0.868625,0.974014
